In [ ]:
%%bash
# pip install tflite-runtime
python3 -m pip install --extra-index-url https://google-coral.github.io/py-repo/ pycoral~=2.0

In [ ]:
import os
import sys
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import tensorflow as tf
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)
from object_detection_ign.satellite_view import WMTSClient, SatelliteView
from object_detection_ign.inference_helpers import decode_img, draw_bounding_boxes_on_image, load_coral_tpus, filter_predictions
from logzero import logger

import ipywidgets as widgets
# from tflite_runtime import interpreter

In [ ]:
# WMTS_SERVICE_URL = "https://wxs.ign.fr/satellite/geoportail/wmts"
WMTS_SERVICE_URL = "https://wxs.ign.fr/ortho/geoportail/wmts?SERVICE=WMTS"
CORRESPONDANCE_TABLE_URL = "https://developers.arcgis.com/documentation/mapping-apis-and-services/reference/zoom-levels-and-scale/"
DATA_PATH = os.path.join("..","data")
CORRESPONDANCE_TABLE_PATH = os.path.join(DATA_PATH, "correspondance_table.csv")
MODEL_PATH = os.path.join('..', 'models', 'tflite_detector_v2', 'model.tflite')
TEST_IMG_DIR = os.path.join("..", "data", "vedai_corrected", "images")
CLASSES_DICT = {0: "background",
                1: "car",
                2: "truck",
                3: "pickup",
                4: "tractor",
                5: "camping car",
                6: "boat",
                7: "motorcycle",
                8: "bus",
                9: "van",
                10: "other",
                11: "small plane",
                12: "large plane"}
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
chosen_image_widget = widgets.Dropdown(
    options=os.listdir(TEST_IMG_DIR),
    # value='2',
    description='Choose a test picture',
    # disabled=False,
)
display(chosen_image_widget)


In [ ]:
test_img = os.path.join(TEST_IMG_DIR, chosen_image_widget.value)

client = WMTSClient(
    WMTS_SERVICE_URL, CORRESPONDANCE_TABLE_PATH, CORRESPONDANCE_TABLE_URL
)

print(client.list_available_zoom_options())
print(client.list_available_layers())
# satellite_view = client.create_satellite_view_from_address(
#     "Aérodrome de Carpiquet", "HR.ORTHOIMAGERY.ORTHOPHOTOS", 18)
satellite_view = client.create_satellite_view_from_position(
    49.408159740153955, 2.782738499678278, "HR.ORTHOIMAGERY.ORTHOPHOTOS", 19)

satellite_view.show_image()
satellite_view.image.save(os.path.join(DATA_PATH, "test_img.jpg"))

In [ ]:
test_img = os.path.join(DATA_PATH, "test_img.jpg")



satellite_detector = tf.lite.Interpreter(MODEL_PATH, experimental_delegates=load_coral_tpus())
satellite_detector.allocate_tensors() 
signature = satellite_detector.get_signature_runner()
input_img_shape = satellite_detector.get_input_details()[0]["shape_signature"][1:3]
input_img_width, input_img_height = input_img_shape[0], input_img_shape[1]
logger.info(f"The selected model uses a {(input_img_width, input_img_height)} size for input images.")
tf_img = decode_img(test_img, img_width=input_img_width, img_height=input_img_height, convert_to_dtype=tf.float32)
output = signature(images=tf_img)
scores, labels, bounding_boxes = filter_predictions(output, CLASSES_DICT)

In [ ]:



# print(scores)
loaded_image = Image.open(test_img)
draw_bounding_boxes_on_image(loaded_image, bounding_boxes, scores, labels)
loaded_image.show()

In [ ]:
# loaded_image = Image.open(TEST_IMG_PATH)
# # loaded_image = loaded_image.resize((input_img_width, input_img_height))
# draw_bounding_boxes_on_image(loaded_image, boxes=output["output_3"].squeeze(), color='red', thickness=2, display_str_list_list=labels)
# loaded_image.show()

In [ ]:
loaded_image.save()

In [ ]:
print(output.keys())
# Output 0 : nb detections
# Output 1 : scores
# Output 2 : classes
# Output 3 : bounding boxes
print(output["output_3"])